In [1]:
import math
import numpy as np
from typing import List

In [2]:
import sys

sys.setrecursionlimit(10**6) 


In [3]:
import requests

# reads an uri and returns its content split by line, deals with \n and \r
def read_uri(uri):
  response = requests.get(uri)
  content = response.text.strip().replace("\r", "").split("\n")
  return content

In [4]:
def dp_change(money, coins):
  # [0  1  2  3  4  5  6  7  8  9 10]
  # [1 0 0 0 0 0 0 0 0 0 0]
  # [0 1 0 1 0 0 0 0 0 0 0]
  # [0 0 2 0 2 0 2 0 0 0 0]
  # [0 0 0 3] i don't care about this situation
  # [0 0 0 0 4]
  possibles = [math.inf] * (money+1)
  possibles[0] = 1
  # assuming its always possible as if there is always a 1 coin denomination
  for j in range(len(possibles)):
    if possibles[j]:
      for coin in coins:
        if j + coin <= money:
          possibles[j + coin] = min(possibles[j] + 1, possibles[j + coin])
  return possibles[money] - 1

In [5]:
print(dp_change(40,[50,25,20,10,5,1]))

2


In [6]:
dp_change(17028, [14,5,3,1]) # 1218

1218

In [7]:
def list_string_to_ints(s):
  return list(map(int, s.split()))

def read_manhattan(uri):
  content = read_uri(uri)
  n, m = list(map(int, content[0].split()))
  down = np.array(list(map(list_string_to_ints, content[1:n+1])))
  right = np.array(list(map(list_string_to_ints, content[n+2:2*n+3])))
  print(down, right)
  return n, m, down, right

In [8]:
read_manhattan("https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/datasets/manhattan.txt")

[[1 0 2 4 3]
 [4 6 5 2 1]
 [4 4 5 2 1]
 [5 6 8 5 3]] [[3 2 4 0]
 [3 2 4 2]
 [0 7 3 3]
 [3 3 0 2]
 [1 3 2 2]]


(4, 4, array([[1, 0, 2, 4, 3],
        [4, 6, 5, 2, 1],
        [4, 4, 5, 2, 1],
        [5, 6, 8, 5, 3]]), array([[3, 2, 4, 0],
        [3, 2, 4, 2],
        [0, 7, 3, 3],
        [3, 3, 0, 2],
        [1, 3, 2, 2]]))

In [9]:
def longest_path(n, m, down, right):

  # print(n, m, len(down), len(right))

  maximal = [0] * (m+1)
  for j in range(1, m + 1):
    maximal[j] = maximal[j - 1] + right[0][j - 1]

  # print(maximal)

  for i in range(1, n+1):
    new_maximal = [0] * (m + 1)
    new_maximal[0] = maximal[0] + down[i - 1][0]
    for j in range(1, m+1):
      up = maximal[j] + down[i - 1][j]
      left = new_maximal[j-1] + right[i][j - 1]
      new_maximal[j] = max(up, left)
    maximal = new_maximal
  return maximal[m]

In [10]:
longest_path(*read_manhattan("https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/datasets/manhattan.txt"))

[[1 0 2 4 3]
 [4 6 5 2 1]
 [4 4 5 2 1]
 [5 6 8 5 3]] [[3 2 4 0]
 [3 2 4 2]
 [0 7 3 3]
 [3 3 0 2]
 [1 3 2 2]]


34

In [11]:
longest_path(*read_manhattan("https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/datasets/dataset_261_10.txt"))

[[4 3 3 1 3]
 [0 2 3 0 4]
 [3 1 2 2 4]
 [4 4 2 0 0]
 [1 2 1 4 0]
 [1 1 0 4 1]
 [1 0 1 2 0]
 [4 1 1 3 0]
 [4 2 0 1 3]
 [1 0 2 0 1]] [[3 4 0 0]
 [3 1 4 3]
 [4 1 1 1]
 [0 3 3 2]
 [4 4 1 0]
 [0 0 1 4]
 [3 0 2 3]
 [0 2 4 1]
 [3 1 4 3]
 [3 3 4 3]
 [2 1 4 2]]


40

In [12]:
def lcs(v, w):
  lv = len(v)
  lw = len(w)
  s = np.zeros((lv + 1, lw + 1))
  backtrack = np.zeros(s.shape)
  for i in range(1, lv + 1):
    for j in range(1, lw + 1):
      matches = v[i-1] == w[j-1]
      s[i, j] = max(s[i-1,j], s[i,j-1], s[i-1,j-1] + matches)
      if s[i,j] == s[i-1,j]:
          backtrack[i, j] = 1
      elif s[i, j] == s[i, j-1]:
          backtrack[i, j] = 2
      else:
          backtrack[i, j] = 0
      # print(i, j, matches, v[i-1], w[j-1])
      # print(s, backtrack)
  return s, backtrack

def lcs_output(backtrack, v, i, j):
  # print(i, j)
  if i == 0 or j == 0:
    return ""
  if backtrack[i, j] == 1:
      return lcs_output(backtrack, v, i - 1, j)
  elif backtrack[i, j] == 2:
      return lcs_output(backtrack, v, i, j - 1)
  else:
      return lcs_output(backtrack, v, i - 1, j - 1) + v[i-1]

In [13]:
s, bt = lcs("AACCTTGG", "ACACTGTGA")
print(bt)
lcs_output(bt, "AACCTTGG", len("AACCTTGG"), len("ACACTGTGA"))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 2. 2. 2. 2. 2. 2. 2.]
 [0. 1. 1. 0. 2. 2. 2. 2. 2. 2.]
 [0. 1. 0. 1. 0. 2. 2. 2. 2. 2.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 0. 2. 2. 2. 2.]
 [0. 1. 1. 1. 1. 1. 1. 0. 2. 2.]
 [0. 1. 1. 1. 1. 1. 0. 1. 0. 2.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


'AACTTG'

In [14]:
s, bt = lcs("AA", "AC")
print(s)
print(bt)
lcs_output(bt, "AA", len("AA"), len("AC"))

[[0. 0. 0.]
 [0. 1. 1.]
 [0. 1. 1.]]
[[0. 0. 0.]
 [0. 0. 2.]
 [0. 1. 1.]]


'A'

In [15]:
s1,s2 = "CTTTTACGAGATACTCAGCCACTTCAATCCTGAGGATCGACAGGGTTATTGGCAGACAGGGCAATGCCGTTTTGCCCTGGATCGTACTGGCTTCCCCATTTATACCTCAGACATTTCATGAGTTGTCGACTCTCTCATACTATGTGCAAATTTATCCCCACGATGGGAACGATTACATATACTGGGCAGTGGATTGCGGTAAAATCGGCTGATCTGCAATGATGATTTGAAATTTCGTCTGCCCGGGATCGGTCTCGAAGAGAATTCATAATGCTTAAGCCTATACTTACAGCTATAGTGAGTCAATGAGTTACCTTAGGCTGCCCGTTGGGAGTCTTCTTCATCCTAACAGGCACGTAGCCTTCGAAATCTTTGATCCCTCAACCCTTATGCTAAGCGAAGAATGTTTCAACACGTTTGGTCTTACTCTGCGCTTTGCAGAAGTGGACCTTACGCCTATATCTGACGTACTCCAGTGCAATATTTCCGGGTAGCCCGGCTAAGTATTCACAGACAAAGTAGGCTTATGGATGCATACAAGAGTCCACCGATCGGTCATTGTATGAAGCAAACTTGTGCGTCAAAGTTCATGAATAGTCACATAATGCAAATGCACCAAAGCATCTCGTAGCAATGTCTGATGAGTAGAATAGGGCGAGGGAGGGGCGCACCTACGGTTTCTAGTCAATGCAGTACAGAAGGAGGAATATGAGGACTCCTTGAGAGAGATGGTAAGGGTCTTTAGACTGAAGTATCTTTTGAAACCGCCATCGGCAGGTTATTCGCTACCGGAAGGGCGACGTAGAAACGTAGTTTCATGGGTAGTAACTGCATAAATAATCGTTCTAATAACCTCTGGAGATTACATTTCACTCCATGAATTGGTGCTGATAGGTGTGTAACCCCAGCAGAGGGTG", "GACTACGATCAAGTCAGGTCCCAGAGCCTAGCATTGTCCCTTTTAGCAGCTGCGAATAATAGACCGCTTGCCTTTATCTCCCCCCGTACGGGGCCTAGTGCAATGAAAACTAGGCTTAAGGACCCTCAGTAAACACCGGTGTCATATTACACGGGCTCAAGTCGGCTTGTCAGCCAAAAAGTACTGGGACTCGAGGCTGCTCTCAATCTAGCAACGTCGCTTCGCGATATTTACGAAATTCGTCCTAGTCACATGATGGCAAGCCATAAGCGTCGAGTTAAAACTACCACGCTTGTCTAGCGCCATTGCTGTCCATGATACATACAAGGTAGTTCCATTGCATTGACTGAATAGAACAGTCAACGGATTCTTTTGCGACCGGCGGACCTAGAGCATAGGAGGGAGTGTATCCAGAAGCCACCGCGTTCAGGTAGTGGTGAGGTTGCTATTACAAGCATGTTCGGTGGGAGGAGGATTGTTTCCACGAAACCACTCGCCCCTCAAGCCCCGGGAACGATGACTAGGTGGATCTAGACAGTCAGGGTAATCAGAGTACTGAAATACGGAACAAATAGAGGGGGACAATCATGGCGGCGGGAGAGCGTAGCTTACAGGTACCAGTTCACGGTTATTTCCAACATGGTGGCGCTAAAACTCACTCGCCTTGATTCTGTAGCTGCCGTCGCAAAACTGAACCGTTGCGAGATCAGTTATGGCGCATGAAATGTTGCATATGTTTATAAGCCGTCAACTGCAAGTCAAGACCAATTGTTACTTCCCCTCTGCGAGTCGACTCAAACAGAACAGTTCGCAGAGATAACCGTACAAGGCCGGATATTGGCACCCGGTTAAGCCATTGATGGGACTTATAAACGTCCTCTAACGCCACCCCCGCATACTGAATATTTTAAAGGGCGTTGACAACATCGAACGACCTGCTTTAGATAGCATTGATCTGGTGCTTGCATTTTTTTAA"
print(len(s1), len(s2))
s, bt = lcs(s1, s2)
lcs_output(bt, s1, len(s1), len(s2))

917 976


'CTACGATATCAGCCCAACCTAGATGCTTTTGCAGCGGAATGCCGTTGCCTATCGTACGGCTTCAATACTAGCTTAAGGACCCTCATACATGTCAATTACCCCAGTGGCGTACAATACTGGGCAGTGATTGCAATCGCTTCGCGATATTTGAAATTCGTCTGCCGATGGCAAGAAATCATAACTAAGCTTCTACGCATGTGTCATGATACTACGGTAGTCTTCTTACTAAAGACAGTCAAATCTTTGACCCACCTAGCTAAGGAGTGTTCAAAGCACCGCGTTCAGAGTGGATTGCTATACGAGTTCGTGAAATTTCCGGTAGCCCGGCTAAGTATTAGACAGTAGGTAATCAGAGTACGATCGGCATGAGACAAATGGCGCAAAGTTCATAAGTCACATAATGCAAATCACCGCATCTGTAGCTGTCGATGAGTAGAATAGGGCAGAGGCATAGTTTTAGTCAATGCAGTCAAGAAATTGACTCCTTGGAGGATAACAGACTGAAGATAACCGCAGGCGGTATTGCACCGGAAGGGGACTAAAACGTTCTAGACGCATAAATATTTTAATAACTCGAGATCTTTATCATGATTGGTGCTGATTTTAA'

In [16]:
s1,s2 = "ACCGTCTTAGCGATCAACACATTTAACAACGCGCCGCACCCCCCGTCAAACGAGCTTTTGGGCTCTTGTCCTTTTACAAGCTTCACGACGCATACAGCCTTGATCAACGGTTTGATCTGTCTCCCTTCAGCTGGCTTTAAAGGACATACATATGAAGGCCTTAATAAGGTCCGGGAACTCCACATATTCGGTACTGGGCAAACCCCATGAACCACCTCAACATGAAGAGTCCGAGGACTCTCACGATCCACCAATGCAGATCGGAACTGTGCGATCGCGTAATGAGCCGAGTACTTGGTTTGTGTTTAGGTTATGGGGGCCGGGAGCCGGTTCAATATAAGGAAGTAGTTGCAGATTAGTTGTTGCGAACGGTCATAAATTTGATGGGTAAACGTGAACTTAACAAACCGTGATAGCTAATCCTATGCATCCCTTACGTGGATCGACTCGAGTACCCAGGTGAACCGACTACTTGATAACCGGAAATCGCGGTATAAAAGCGCTCACGGTCAGGAGATATACCTCCAAGCAGTAGTCTTTCTGAGCCTAGAGTAGTAAATTACAGGGACGATGTCTTTTACCGAGGCAACATTTTATTGAGAATCACATGAGGCACAGGTAAAGGCGACATCACGATCGAGATCAACCCCTACTTGTTCAAAACATTGAGAACCAGCTCTGTTTTGGAACCTAGAAAGATAACGCATCCGCTTGATATTCCACGGCTTGTCCCTCTTGTGCGGTCCATCTATCGGAGTTTCCTCCGATACGACCCGCAATGTTTCCAGGCGTACGGTACTTTATGAATACACTCGCGCTGTAACCTGTTATGTGAAACACACACGACAGAGCTTCGCGTGGGCCCAGCGACCCGGTAATACTACATCACCGCACACGACCTCGAGCAGTCTTTGCCGGCGTCCGTAAGTAGTCTAAAGTTGTGTTGATGCTTGGGGTTAAAGCTAAATCGTCCGCAGAATACGACTCTCATCCCAAT","ACCCGCACGCGCTTTGGTCTAGATTCTAGCTCCAACTTGCCTGCTAGATACTCTGTTAAAAGATGGTTTTACAACCCCCTCCTCTGTCCCTGGGGTATTATATAATACGTCGGATAGTCAGGTACAAATACAAGTGGGTGGGAATACTTTTCCTCGGATCCTAGACCACGGATTACTGCGTGGTTGACAAGAGTCGGCCCGGAGGGAAACGTGAAGGTTAGTGCAATTAAAGTCTCTAATGTGAAGCCTCCGCGAAGCGAGGAGTTTCTGAGATCGAGTACTATTTAGAGTTCGAAATCACGGCTTAACCTCACTGCCACGCATAACTTGCCGGCAATCCAGTTTTGCAACGATACTTAATTTGTGCAGCTCATCTTTGCTGTCCAGAAATAGAGCTAGTCGATCTCATCTTGCGGGTAGCCAGAAGTCCTACCGTCTCCTCCATGTAGCTTAAAAATTTCGGTGAGGATCAAAAATGATAAACGTGACAGGTAAGCTCCTACGTCTATCCTATGACCCCCGCGGCAGAATAGGTTGGTAGTGTTAGTGCGTGAGCTGGTAGAATAGAGCACACTTAGGGAAACGGGAACCGTTATGTAGGGCTGCGACACACAAAAAAGTGTTCGTTGGTAAGCTGCCTCTCCACTAAACAGGATTTCTCTGGATGATCCCATCGAAGCAAGTTACGCACCACGCCGAGGCGGACCCTGGTACTAGCTGCCCCCCCCTTTATGGGGCGCTCGTACATCAAGATGATCGCGGACTCAACCTGATTACGAGTTGTCCAAGTAGTCCAGGGTAAGAGAAACTGGAGAGA"
s, bt = lcs(s1, s2)
len(lcs_output(bt, s1, len(s1), len(s2)))

573

In [17]:
len("ACCGCAGCGTCAATTTACAACGCCGCACCGTAAAGATGGTTTTACAACCCCCTCCCTGTCCGGTTTATTTCTCTAGTCAGGACAAATAAAGTGGTGGGAATACTTTCTCGGACCAGACCACTACTGGTGGTTGACAAGAGTCGGCCCGGAGGGAACTGGTTGTGTTAGTTATGGGCCCCGGAAGGAGAGTTGAGATCGAGTCTATTTGAGTCGAATCACGGCTAACCTATGCACCTACTTGCCGATCCAGTGAACGATACTTATACCATCGCGTAAAAAGGCTAGTCGATATCCTCCAGAGTAGTCTTCTGAGCTAAAAATTCGGGAGATCAAAAATATAAACTGACAGGTAAGCCTACGTCATCAACCCCCGCAAAATTGGAGTGTTTTGGCTAGAAAGAGCACCTTGAAACGGGCCTTTGTGGGTCCACACAGTTTCTGTAAGCTGTTCCACTACGGTCTTATGATCATCGGCAAGTTAGCACCACGCGAGGCGGACCCGGTACTACTCCCCCACGCTCGACATCTTGCGGCTCCTGATTAAGTTGTGTGTCGGGTAAAGAAACTGAGAGA")

573

In [18]:
s1, s2 = "GACT","ATG"
s, bt = lcs(s1, s2)
r = lcs_output(bt, s1, len(s1), len(s2))
print(len(r), r)

2 AT


In [19]:
s1, s2 = "ATG", "GACT"
s, bt = lcs(s1, s2)
print(s, bt)
r = lcs_output(bt, s1, len(s1), len(s2))
print(len(r), r)

[[0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1.]
 [0. 0. 1. 1. 2.]
 [0. 1. 1. 1. 2.]] [[0. 0. 0. 0. 0.]
 [0. 1. 0. 2. 2.]
 [0. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1.]]
2 AT


In [20]:
s1, s2 = "AGAGTAGACACCTTGCGCTCTGTTTGATATGGAAGCTCGCAGGTCCAGAGTGAAATCATAGAAATTACCATAAGCGATGATATGAAGCACAGATCACTGTTTGTGCTCAGTGGCATCGACTCTATTTAAACATAGTATACCGGTTTATTATAGGTCCTGTGCCTGGCCTCGTGTGCCGTGCCTTGGGATTTGAACGCTGAACTACCCACGCGCACGGAAGGGGTTCCAGTGCGCTAACAATAACACTGTATCCCAGTCGAGTCTGATCGCCCTAGAAAAGCAATTGCACTCAGGGAAACGTGCGGCGGCTCGGTCATCTCGATTGACGTGGGGTCCGCCGCTGAATTTAAAGACAGGCTATGATTGTTCCTTAACCGATCAAGCATGCGATTATTGGTCCCCTATACTCTATTACGTAATGCGGTGATCTGGTACTGGCAGCTGTCGATCTCGTGAATCTCCCGCTGCATTATTCGCCGTACTACCGAATGAGAGTGATCCCCGCCGGGCACTGCACCTGTGCGCCGGAGAATCGAGTTCATCACAACCACTGCTACAGACCTCTATAGACCGTAGACCGACCCCACGGTCCGTTATGAAGGAAGTCCAAAAGAGGCGGATGTCCTACGCTCTCTTCGCAGATCTATTCATACCTCCGGGTAAATTCTATGACAGCCCTACCACGGGATATGTACACTCTCACATGGAGATTTTTAGGTACAGATATTCATTGATGTACAATGGGCGGAGCGCCAGCGGACCGACCGAAATATCGAGTGTCGAGGATCAATCACGTTGCTACGTGCACCCGGCACCCGACAAATTCACGTGCAGCCGTCTCTTGGTAAGCGGGCACCGTCACGGATCGTGCGTGCCTAGCCTAGTGCCCGTGGGGCTATTGCTGGGGTATCCCCTTGGGAGAAGTTACGACGTTGATGATCCC", "CAGGAAGTGGTAGATAACCACTAGCTGACCAAGCAACCAGCATTCATACAACTTATTTAGGGAAGAGAAGGAGCAAGGTGTGGAGAAGTCATACCCCGTATGGATACGTTATTTGGGTCCAGAAATTAGTATAGGTGCCAACGAACAATTTCGATTCAATTAGATAGCGGTTCGCACCTCCTTCGGAGTGTGTGCGCAGCAGCTATCTTGCCCGCAGAAAGAACGCGGTGTCCCTGGGCTAACCGGGATGGTAAGCATTTTATAATCTGTGCACACGTCTATCATGAGACCTCTTCATGCCCGGTCGAAGGCGTAGGGATCAGCGGCTGGGATGGCCAAGTGCACGCCTTTTATGAGGGGATCCACTCTACTTGCGATCGCAACGTGCTGGGGTACGGCAACGTCCCCAGTAGGCTCGTTGCTGTACACGACTGCAGGTACTTGTTGAGGCAGGGCGCCCAATATAGTGGTCTGGACTCCGGCCCCATGATATTGATTATCACGTCATCGTGCCAACCTGAAACGGACTTTAGCTAGATGCTGGTCTGGTAGAATCAGAGATCTCCTTGTCCGTCTGTAAATCAGTGCAATCACGGAAATGTTGACACTATGTATGATAATCTGTTTCGAAACCGAACACGGTACACGGCTACGTACACTCGGTGTGTACATCAGCCTCAGCGCCCGAATCTCTGCGAAGATCAATCCTTGACCTCGGTGATACTTCGCTTAAATGTCGCGATACACAGCGTTCATAGAATAAGTAAACATACGCTGTAAGCTGGATAAGAAGAATTTTAGAAACTCAA"
s, bt = lcs(s1, s2)
r = lcs_output(bt, s1, len(s1), len(s2))
print(len(r), r)

568 AGAGTAGACACTGCGCCAAGAACCGCATCATAAATATAGAAAAAAGCGTGTAGAAGCACAGATACGTTTTTGGGCAGATTATTTAAACAATTCGATTAATTGTGCGGTCGCCTCCTTGGGTTTGCGCGACTACCCCGCGAGAAGGGGTCCTGGCTAACATAACATTATCCCAGTCATCTGAGCCCTCATGCCCGGGAACGTGGGCGGCTGGATCCAAGTGCCGCCTTTTAGAGGGATCCTTACCGATCAAGCTGCGAAGTCCCCATACTCGTTGCGTACGACTGCAGGTACTTGTTGGCAGCGCCCAATAAGTGTCGGACTCCGGCCCAGAATGATTATCACCACTGCCAACCTCATAGCTAGAGCGGTCGTAGAATCAGAGATTCCTGTCTCTTAATCATCAATCCGGAAATTTGACACTAGTATGTAATCTCAAAGAAGGTACAGATACATTGTGTACATCAGCCCAGCGCCCGAATTCTGCGAGATCAATCCTTGCTCGTGAACCGCAAATTCCGTCAGCGTTCTGTAAGTACATCGCTGTAAGCTGGTAGGATTTAGAAACCAA


In [21]:
r

'AGAGTAGACACTGCGCCAAGAACCGCATCATAAATATAGAAAAAAGCGTGTAGAAGCACAGATACGTTTTTGGGCAGATTATTTAAACAATTCGATTAATTGTGCGGTCGCCTCCTTGGGTTTGCGCGACTACCCCGCGAGAAGGGGTCCTGGCTAACATAACATTATCCCAGTCATCTGAGCCCTCATGCCCGGGAACGTGGGCGGCTGGATCCAAGTGCCGCCTTTTAGAGGGATCCTTACCGATCAAGCTGCGAAGTCCCCATACTCGTTGCGTACGACTGCAGGTACTTGTTGGCAGCGCCCAATAAGTGTCGGACTCCGGCCCAGAATGATTATCACCACTGCCAACCTCATAGCTAGAGCGGTCGTAGAATCAGAGATTCCTGTCTCTTAATCATCAATCCGGAAATTTGACACTAGTATGTAATCTCAAAGAAGGTACAGATACATTGTGTACATCAGCCCAGCGCCCGAATTCTGCGAGATCAATCCTTGCTCGTGAACCGCAAATTCCGTCAGCGTTCTGTAAGTACATCGCTGTAAGCTGGTAGGATTTAGAAACCAA'

In [22]:
class Path:
  def __init__(self, path):
    self.path = path

  def __repr__(self):
    return "->".join(map(str,self.path))

In [23]:
from collections import defaultdict

class Graph:

  def __init__(self):
    self.adjacency = defaultdict(dict)
    self.all_nodes = set()

  def connect(self, a, b, w):
    self.adjacency[a][b] = w
    self.all_nodes.add(a)
    self.all_nodes.add(b)

  def __topological_dfs(self, current, stack, visited):
    if current in visited:
      return stack
    visited.add(current)
    neighboors = self.adjacency[current].keys()
    for node in neighboors:
      self.__topological_dfs(node, stack, visited)
    stack.append(current)
    return stack

  # unfortunately, there is no need for all N to be on the [0,...,N-1] interval :(  
  def __max_nodes_required(self):
    return (max(self.all_nodes) + 1)


  def components(self):
    components = [0] * self.__max_nodes_required()
    for i in self.all_nodes:
      components[i] = i
    # can be implemented faster with references, this is O(n^3)
    for i in self.all_nodes:
      for j in self.adjacency[i].keys():
        for k in self.all_nodes:
          if components[k] == j:
            components[k] = components[i]
    return components

  def component_containing(self, key):
    components = self.components()
    component = {i for i in self.all_nodes if components[i] == components[key]}
    return component

  def __longest_paths(self, starting: int):
    component = self.component_containing(starting)
    topological = self.topological()
    topological_on_path = [i for i in topological if i in component]
    max_distance = [0] * self.__max_nodes_required()
    previous = [-1] * self.__max_nodes_required()
    appeared = False
    for a in topological:
      if a == starting:
        appeared = True
      if a in component and appeared:
        for b, w in self.adjacency[a].items():
          new_distance = max_distance[a] + w
          if new_distance > max_distance[b]:
            max_distance[b] = new_distance
            previous[b] = a
    return max_distance, previous

  # DFS implementation, nasty...
  def longest_path_to(self, starting: int, ending: int):
    max_distance, previous = self.__longest_paths(starting)
    return self.__recreate_longest_path(starting, max_distance, previous, ending)

  def __recreate_longest_path(self, starting: int, max_distance: List, previous: List, ending: int):
    path = [ending]
    current = ending
    while current != starting:
      current = previous[current]
      path.append(current)
    path = list(reversed(path))
    return max_distance[ending], Path(path)


  # DFS implementation, nasty...
  def longest_path(self, starting: int):
    max_distance, previous = self.__longest_paths(starting)
    ending = np.argmax(max_distance)
    return self.__recreate_longest_path(starting, max_distance, previous, ending)


  def topological(self):
    stack = []
    visited = set()
    for i in self.all_nodes:
      self.__topological_dfs(i, stack, visited)
    return list(reversed(stack))

  def __repr__(self):
    return f"{self.all_nodes}:::{self.adjacency.__repr__()}"

In [24]:
def adjancecy_string_to_graph(adj_string: str):
  graph = Graph()
  for line in adj_string.split("\n"):
    a = int(line.split("->")[0])
    b = int(line.split("->")[1].split(":")[0])
    w = int(line.split(":")[1])
    graph.connect(a, b, w)
  return graph

In [25]:
g = adjancecy_string_to_graph("""0->1:7
0->2:4
2->3:2
1->4:1
3->4:3
5->6:10""")
g

{0, 1, 2, 3, 4, 5, 6}:::defaultdict(<class 'dict'>, {0: {1: 7, 2: 4}, 2: {3: 2}, 1: {4: 1}, 3: {4: 3}, 5: {6: 10}})

In [26]:
g.topological()

[5, 6, 0, 2, 3, 1, 4]

In [27]:
g.component_containing(0)

{0, 1, 2, 3, 4}

In [28]:
g.longest_path_to(0, 4)

(9, 0->2->3->4)

In [29]:
g = adjancecy_string_to_graph("""0->6:11
6->12:11
12->30:1
30->49:10
7->47:3
15->42:19
48->49:20
42->49:9
23->43:1
18->42:4
37->42:2
38->47:2
28->41:15
18->33:10
0->42:6
1->49:16
16->31:18
32->35:4
26->44:10
7->40:11
25->28:15
34->41:1
31->42:5
25->42:17
17->34:10
22->27:13
17->22:16
19->35:15
24->35:18
36->47:19
25->49:18
15->29:15
17->21:6
10->33:6
0->47:14
5->20:2
7->43:4
40->41:13
12->35:16
30->42:4
16->39:2
24->48:2
23->31:7
16->20:4
20->29:19
46->49:4
5->28:12
25->37:7
35->41:11
45->47:1
14->15:14
0->35:7
11->32:12
36->41:12
3->27:11
29->40:17
22->33:8
19->23:20
21->30:11
33->44:15
17->47:9
8->26:13
18->19:14
3->40:12
46->47:19
41->47:16
13->31:18
1->18:8
32->41:2
32->38:13
11->16:18
27->49:3
44->49:9
4->41:16
13->15:17
39->41:2
10->32:9
37->39:16
3->16:4
8->28:2
4->23:20
41->48:16
18->28:15
14->34:12
18->44:18
7->8:12
31->41:11
5->49:9
0->26:16
38->42:16
0->1:7
1->3:18
3->4:15
4->5:19
5->7:10
7->10:17
10->11:8
11->13:8
13->14:12
14->17:12
17->24:16
24->25:14
25->36:18
36->45:16
45->46:15""")
g.longest_path_to(0, 49)

(222, 0->1->3->4->5->7->10->11->13->14->17->24->25->36->41->48->49)

In [30]:
s, bt = lcs("GCGATC", "CTGACG")
print(bt)
lcs_output(bt, "GCGATC", len("GCGATC"), len("CTGACG"))

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 2. 2. 2.]
 [0. 0. 2. 1. 1. 0. 2.]
 [0. 1. 1. 0. 2. 1. 0.]
 [0. 1. 1. 1. 0. 2. 1.]
 [0. 1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 0. 2.]]


'CGAC'

In [31]:
s, bt = lcs("CTCGAT", "TACGTC")
print(bt)
lcs_output(bt, "CTCGAT", len("CTCGAT"), len("TACGTC")) 

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 2. 2. 2.]
 [0. 0. 2. 1. 1. 0. 2.]
 [0. 1. 1. 0. 2. 1. 0.]
 [0. 1. 1. 1. 0. 2. 1.]
 [0. 1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 0. 2.]]


'TCGT'

In [32]:
from itertools import combinations_with_replacement
from itertools import permutations
def combinations_for_masses(m1, m2, target):
  possibles = 0
  for length in range(math.ceil(target / min(m1,m2)) + 1):
    for comb in combinations_with_replacement([m1, m2], length):
      if sum(comb) == target:
        print(comb)
        # very slow, could be optimized because we only need the number
        possibles += len(set(permutations(comb)))
  return possibles

In [33]:
combinations_for_masses(2,5,10)

(5, 5)
(2, 2, 2, 2, 2)


2

In [34]:
# combinations_for_masses(2,3,25)

In [35]:
# combinations_for_masses(2,3,22)

In [36]:
g = adjancecy_string_to_graph("""1->2:0
1->3:0
1->4:0
1->5:0
1->6:0
2->3:0
2->6:0
3->4:0
5->4:0
5->6:0""")
g.topological() == [1, 5, 2, 6, 3, 4]

True

In [37]:
import re

def read_cost_table(name):
  uri = f"https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/{name}.txt"
  data = read_uri(uri)
  headers = re.split("\s+", data[0])
  costs = defaultdict(dict)
  for line in data[1:]:
    line_data = re.split("\s+", line)
    line_name = line_data[0]
    for i, col_value in enumerate(line_data[1:]):
      col_name = headers[i]
      costs[line_name][col_name] = int(col_value)
  return costs

read_cost_table("BLOSUM62")['A']['A'] == 4

True

In [93]:
class CostPath:

  def __init__(self, first, second, backtrack, x = None, y = None, finalize_on_left=False, finalize_on_top=False):
    self.backtrack = backtrack
    self.first= first
    self.second = second
    self.start_x = x
    if x == None:
      self.start_x = len(self.first.code)
    self.start_y = y
    if y == None:
      self.start_y = len(self.second.code)
    self.finalize_on_left = finalize_on_left
    self.finalize_on_top = finalize_on_top

  def __repr__(self):
    return self.__output(self.backtrack, self.start_x, self.start_y)

  def __output(self,backtrack, i, j):
    first_stack = ""
    second_stack = ""
    while (i != 0 or j != 0):
      # print(i, j, backtrack[i,j])
      if backtrack[i, j] == 1:
        first_stack += self.first.code[i-1]
        second_stack += "-"
        i -= 1
      elif backtrack[i, j] == 2:
        second_stack += self.second.code[j-1]
        first_stack += "-"
        j -= 1
      elif backtrack[i, j] == 0:
        first_stack += self.first.code[i-1]
        second_stack += self.second.code[j-1]
        i -= 1
        j -= 1
      else:
        i = 0
        j = 0
      if self.finalize_on_left and j==0:
        break
    return "".join(reversed(first_stack)) + "\n" + "".join(reversed(second_stack))


In [97]:
from typing import Dict

class Peptide:


  def __init__(self, code):
    self.code = code


  def global_alignment(self, other_peptide, cost_matrix: Dict, indel_penalty=5):
    # can be optimized to use only 2 lines if no backtrack is required
    li = len(self.code)
    lj = len(other_peptide.code)
    scores = np.zeros((li+1, lj+1))
    for i in range(1, lj+1):
      scores[0,i] = scores[0,i-1] - indel_penalty
    for i in range(1, li+1):
      scores[i,0] = scores[i-1, 0] -indel_penalty
    backtrack = np.zeros(scores.shape)
    backtrack[0,:] = 2
    backtrack[:,0] = 1
    for i in range(1, li + 1):
      for j in range(1, lj + 1):
        from_top = scores[i-1,j] - indel_penalty
        from_left = scores[i,j-1] - indel_penalty
        extra_cost = cost_matrix[self.code[i-1]][other_peptide.code[j-1]]
        from_diagonal = scores[i-1,j-1] + extra_cost
        scores[i, j] = max(from_top, from_left, from_diagonal)
        # print(i, j, scores[i,j], from_top, from_left, from_diagonal, extra_cost, self.code[i-1], other_peptide.code[j-1])
        if scores[i, j] == from_top:
            backtrack[i, j] = 1
        elif scores[i, j] == from_left:
            backtrack[i, j] = 2
        else:
            backtrack[i, j] = 0
    return scores, CostPath(self, other_peptide, backtrack)

  def local_alignment(self, other_peptide, cost_matrix: Dict, indel_penalty=5):
    li = len(self.code)
    lj = len(other_peptide.code)
    scores = np.zeros((li+1, lj+1))
    backtrack = np.zeros(scores.shape)
    backtrack[0,:] = 2
    backtrack[:,0] = 1
    for i in range(1, li + 1):
      for j in range(1, lj + 1):
        from_beginning = 0
        from_top = scores[i-1,j] - indel_penalty
        from_left = scores[i,j-1] - indel_penalty
        extra_cost = cost_matrix[self.code[i-1]][other_peptide.code[j-1]]
        from_diagonal = scores[i-1,j-1] + extra_cost
        scores[i, j] = max(from_top, from_left, from_diagonal, from_beginning)
        # print(i, j, scores[i,j], from_top, from_left, from_diagonal, from_beginning, extra_cost, self.code[i-1], other_peptide.code[j-1])
        if scores[i, j] == from_top:
            backtrack[i, j] = 1
        elif scores[i, j] == from_left:
            backtrack[i, j] = 2
        elif scores[i, j] == from_diagonal:
            backtrack[i, j] = 0
        else:
            backtrack[i, j] = 3
    x, y = np.unravel_index(scores.argmax(), backtrack.shape)
    return scores, CostPath(self, other_peptide, backtrack, x, y)


  def edit_distance(self, other_peptide):
    li = len(self.code)
    lj = len(other_peptide.code)
    scores = np.zeros((li+1, lj+1))
    scores[:,0] = range(len(self.code) + 1)
    scores[0,:] = range(len(other_peptide.code) + 1)
    for i in range(1, li + 1):
      for j in range(1, lj + 1):
        differs = self.code[i-1] != other_peptide.code[j-1]
        from_diagonal = scores[i-1][j-1] + differs
        from_top = scores[i-1,j] + 1
        from_left = scores[i,j-1] + 1
        scores[i, j] = min(from_top, from_left, from_diagonal)
        # print(i,j,self.code[i-1] , other_peptide.code[j-1],differs,from_diagonal,from_top,from_left, scores[i,j])
    return scores

  def fitting_alignment(self, other_peptide, indel_penalty = 1, mismatch_penalty = 1, match_bonus = 1):
    scores, backtrack = self.generic_alignment(other_peptide,
                                  indel_penalty=indel_penalty,
                                  mismatch_penalty=mismatch_penalty,
                                  match_bonus=match_bonus)
    maxv = scores[:,-1].argmax()
    i, j = maxv, len(other_peptide.code)
    return scores, backtrack, (i, j), scores[:,-1], scores[:,-1].max(), CostPath(self, other_peptide, backtrack, i, j, finalize_on_left=True)
    
  def overlap_alignment(self, other_peptide, indel_penalty = 1, mismatch_penalty = 1, match_bonus = 1):
    scores, backtrack = self.generic_alignment(other_peptide,
                                  indel_penalty=indel_penalty,
                                  mismatch_penalty=mismatch_penalty,
                                  match_bonus=match_bonus)
    maxv = scores[-1,:].argmax()
    i, j = len(self.code), maxv
    return scores, backtrack, (i, j), scores[-1,:], scores[-1,:].max(), CostPath(self, other_peptide, backtrack, i, j, finalize_on_left=True)

  def generic_alignment(self, other_peptide, indel_penalty = 1, mismatch_penalty = 1, match_bonus = 1):
    li = len(self.code)
    lj = len(other_peptide.code)
    scores = np.zeros((li+1, lj+1))
    scores[0,:] = np.array(list(range(0, -lj-1, -1))) * indel_penalty
    backtrack = np.zeros(scores.shape)
    backtrack[0,:] = 2
    backtrack[:,0] = 1
    for i in range(1, li + 1):
      for j in range(1, lj + 1):
        from_top = scores[i-1,j] - indel_penalty
        from_left = scores[i,j-1] - indel_penalty
        matches = self.code[i-1] == other_peptide.code[j-1]
        if matches:
          extra_cost = match_bonus
        else:
          extra_cost = -1 * mismatch_penalty
        from_diagonal = scores[i-1,j-1] + extra_cost
        scores[i, j] = max(from_top, from_left, from_diagonal)
        # print(i, j, scores[i,j], from_top, from_left, from_diagonal, from_beginning, extra_cost, self.code[i-1], other_peptide.code[j-1])
        if scores[i, j] == from_top:
            backtrack[i, j] = 1
        elif scores[i, j] == from_left:
            backtrack[i, j] = 2
        else:
            backtrack[i, j] = 0
    return scores, backtrack



In [98]:

Peptide("PAWHEAE").overlap_alignment(Peptide("HEAGAWGHEE"), mismatch_penalty=2, indel_penalty=2)

(array([[  0.,  -2.,  -4.,  -6.,  -8., -10., -12., -14., -16., -18., -20.],
        [  0.,  -2.,  -4.,  -6.,  -8., -10., -12., -14., -16., -18., -20.],
        [  0.,  -2.,  -4.,  -3.,  -5.,  -7.,  -9., -11., -13., -15., -17.],
        [  0.,  -2.,  -4.,  -5.,  -5.,  -7.,  -6.,  -8., -10., -12., -14.],
        [  0.,   1.,  -1.,  -3.,  -5.,  -7.,  -8.,  -8.,  -7.,  -9., -11.],
        [  0.,  -1.,   2.,   0.,  -2.,  -4.,  -6.,  -8.,  -9.,  -6.,  -8.],
        [  0.,  -2.,   0.,   3.,   1.,  -1.,  -3.,  -5.,  -7.,  -8.,  -8.],
        [  0.,  -2.,  -1.,   1.,   1.,  -1.,  -3.,  -5.,  -7.,  -6.,  -7.]]),
 array([[1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 2., 1., 0., 2., 0., 2., 2., 2., 2.],
        [1., 0., 2., 2., 2., 2., 1., 0., 0., 2., 2.],
        [1., 1., 0., 2., 2., 2., 2., 2., 1., 0., 2.],
        [1., 2., 1., 0., 2., 2., 2., 2., 2., 1., 0.],
        [1.,

In [100]:

Peptide("ATCTGGCTTAAGACAGAACCCTGCCCTTGGACGCATGTGGTGTAGCTCGTGACGATCAAGATGCCAGTCGCACCACACTGATTAGTGCCTCCATGAAATCAACACTGCATGCGGGGATATAGATATATTCTACGATGCGACGGTGCTCCAGACATCCTTGGGCACGCAGACGGGCCACACACCACGAGTGCTCGTACACCGCCGTTGTGGGCGAAGCGTCCCGTGATATGAATGATGCCTAGTCGATTGTCGCGGGCTCACGAGGCTACAATATAAACAGTGTGGAGGTACTTACAATCGGAGCATAGCCGGTCGTCCGGCAAAAAGGAACCAGATGGCCCTTATGGAAGATCTCGTAACTCGTTACGTAAACTGAGAACAGATGGAACCGAGTTCCTGCCATTGTACCGGCCTTCATCCCCCAAGACGCCGCTGTGCTTCCTAGTCATTTAACGTTCTGTAATTAAAGATCTATAAGATCCCTCGCAATTTTTTGAAATGTGTTTGTCATTAGCAAACGCTGTTATACGTCTGCATAGATTCGGGGATTTGGGTGCTAGAGATGCGCCTCGCGGATTCGCGTAGTCATGGGGCCCTTGTCGGGAAGCTCCACGAAGCGTGATGGCGCTACGCTGGCCCTTGATCCGGCCGAGACATTCGCGATGCCTCAGAGGCTAAATTTTCTCATCAGAGGGTGTGCCGTATCGCGTGGTGATCGGAAAGATATCTCACTACCAAATGACTTTTGGTAATCGATGGGGGGTGATCGGTCTTGAGATAGCACTGTGCCAGTCTGTACAAGGGTAACTTAGGGCCGGCGTGTGTACCCCCATGTCACTATCCACGGTTTGATAATCTCTCCAAAGAGGAGTCTATGATA").overlap_alignment(Peptide("AAGAGTGACAGCATTAGGGCCCGCCGTTGATCCGGTCGACGACATGTACGCATTCAGAGGGCAAATTTGCTGTCAGCAGTTGTCCGTATCCCGTGTGACCGGGAAGACAATCTCACATGAAATACTTCTAGCTGAATCGATGGGGATCTCGGCCTCGAGGTCGCACGCGCACTCCATACAAGGGGTAGCTGGGCGCGACGTGTGGTATACTCCCCCGAGTCAATATCCACGAGTTCGATAACCCCTTCGCCGAGGGAAGTCTAATAATGTCTACGTACGCACACTTTCGTGTGACGGTGCTGATAGTGAGAGGCAACTCCTAAAGGTTCTATGGGCTCCTCACTTTCCCAAGGGCCCTCGCTTCCTTAATCAATAATACTGCCCGTACGGGAGAATTCAGTTTTGGCCACTCATGAGCTCCGACAACTCAGGAGTTACTACTCAGGAAAATAGTATCACGAGTTGAGTCAACCACCGGACCTATCGAGGGCGTAAAGTACATTGCCGTCACAGTTCATCTATAGGCGTGCCGTTGGCAACTTGTATTGGTTTATCGAGCGTCACCTGATATTATTGCCTGCGGCTGCCCACAGATGACCGTGCAGGTTATATCGCGAACGTTACAGAAGGGTCGATCTAGATCTCTCTTCACTTTGAACTTGTGAACCAGTCAATCCGCTGTTGTCTGCACATGCCAAGGAGAAGTCTTCCCGGCCCGCAATTATCCGACACGCGGGAATTCCCTACGATCTGGCGTTCTATGGCTAGCAGGTTAAACCACTGGGCGAGAGTTCTAGCGCAGCTACCGTCCGTGTACCGGTAGGGGGGTTGTTGACACTTGTAAGAGGGGGGTACGAACTGTGCGAGTTATAAGTCTGTCGAACCGCGTCGACCTCGCG"), mismatch_penalty=2, indel_penalty=2)

(array([[ 0.000e+00, -2.000e+00, -4.000e+00, ..., -1.794e+03, -1.796e+03,
         -1.798e+03],
        [ 0.000e+00,  1.000e+00, -1.000e+00, ..., -1.791e+03, -1.793e+03,
         -1.795e+03],
        [ 0.000e+00, -1.000e+00, -1.000e+00, ..., -1.788e+03, -1.790e+03,
         -1.792e+03],
        ...,
        [ 0.000e+00,  1.000e+00, -1.000e+00, ..., -3.480e+02, -3.480e+02,
         -3.500e+02],
        [ 0.000e+00, -1.000e+00, -1.000e+00, ..., -3.500e+02, -3.500e+02,
         -3.500e+02],
        [ 0.000e+00,  1.000e+00,  0.000e+00, ..., -3.510e+02, -3.520e+02,
         -3.520e+02]]), array([[1., 2., 2., ..., 2., 2., 2.],
        [1., 0., 2., ..., 2., 2., 2.],
        [1., 1., 0., ..., 2., 2., 2.],
        ...,
        [1., 0., 2., ..., 1., 0., 1.],
        [1., 1., 0., ..., 1., 1., 0.],
        [1., 0., 0., ..., 2., 1., 1.]]), (880,
  266), array([   0.,    1.,    0.,   -2.,   -2.,   -3.,   -2.,   -2.,   -1.,
          -3.,   -4.,   -6.,   -6.,   -3.,   -2.,   -2.,   -1.,   -3.,
      

In [90]:

Peptide("ACGACAGAG").fitting_alignment(Peptide("CGAGAGGTT"), mismatch_penalty=3, match_bonus=2)

(array([[ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
        [ 0., -1., -2.,  0., -1., -2., -3., -4., -5., -6.],
        [ 0.,  2.,  1.,  0., -1., -2., -3., -4., -5., -6.],
        [ 0.,  1.,  4.,  3.,  2.,  1.,  0., -1., -2., -3.],
        [ 0.,  0.,  3.,  6.,  5.,  4.,  3.,  2.,  1.,  0.],
        [ 0.,  2.,  2.,  5.,  4.,  3.,  2.,  1.,  0., -1.],
        [ 0.,  1.,  1.,  4.,  3.,  6.,  5.,  4.,  3.,  2.],
        [ 0.,  0.,  3.,  3.,  6.,  5.,  8.,  7.,  6.,  5.],
        [ 0., -1.,  2.,  5.,  5.,  8.,  7.,  6.,  5.,  4.],
        [ 0., -1.,  1.,  4.,  7.,  7., 10.,  9.,  8.,  7.]]),
 array([[1., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 2., 0., 2., 2., 2., 2., 2., 2.],
        [1., 0., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 1., 0., 2., 2., 2., 2., 2., 2., 2.],
        [1., 1., 1., 0., 2., 2., 2., 2., 2., 2.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 0., 2., 2., 2., 2.],
        [1., 1., 0., 1., 0., 1., 0., 2., 2., 2.]

In [91]:

Peptide("CAAGACTACTATTAG ").fitting_alignment(Peptide("GG"), match_bonus=10)

(array([[ 0., -1., -2.],
        [ 0., -1., -2.],
        [ 0., -1., -2.],
        [ 0., -1., -2.],
        [ 0., 10.,  9.],
        [ 0.,  9.,  9.],
        [ 0.,  8.,  8.],
        [ 0.,  7.,  7.],
        [ 0.,  6.,  6.],
        [ 0.,  5.,  5.],
        [ 0.,  4.,  4.],
        [ 0.,  3.,  3.],
        [ 0.,  2.,  2.],
        [ 0.,  1.,  1.],
        [ 0.,  0.,  0.],
        [ 0., 10., 10.],
        [ 0.,  9.,  9.]]), array([[1., 2., 2.],
        [1., 2., 2.],
        [1., 2., 2.],
        [1., 2., 2.],
        [1., 0., 2.],
        [1., 1., 0.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 0., 0.],
        [1., 1., 1.]]), (15,
  2), array([-2., -2., -2., -2.,  9.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,
         1.,  0., 10.,  9.]), 10.0, GACTACTATTAG
 G----------G)

In [42]:

Peptide("GAGA").fitting_alignment(Peptide("GAT"), indel_penalty = 2)


(array([[ 0., -2., -4., -6.],
        [ 0.,  1., -1., -3.],
        [ 0., -1.,  2.,  0.],
        [ 0.,  1.,  0.,  1.],
        [ 0., -1.,  2.,  0.]]), array([[1., 2., 2., 2.],
        [1., 0., 2., 2.],
        [1., 1., 0., 2.],
        [1., 0., 1., 0.],
        [1., 1., 0., 2.]]), (3, 3), array([-6., -3.,  0.,  1.,  0.]), 1.0, GAG
 GAT)

In [43]:

Peptide("GTAGGCTTAAGGTTA").fitting_alignment(Peptide("TAGATA"))


(array([[ 0., -1., -2., -3., -4., -5., -6.],
        [ 0., -1., -2., -1., -2., -3., -4.],
        [ 0.,  1.,  0., -1., -2., -1., -2.],
        [ 0.,  0.,  2.,  1.,  0., -1.,  0.],
        [ 0., -1.,  1.,  3.,  2.,  1.,  0.],
        [ 0., -1.,  0.,  2.,  2.,  1.,  0.],
        [ 0., -1., -1.,  1.,  1.,  1.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  2.,  1.],
        [ 0.,  1.,  0., -1., -1.,  1.,  1.],
        [ 0.,  0.,  2.,  1.,  0.,  0.,  2.],
        [ 0., -1.,  1.,  1.,  2.,  1.,  1.],
        [ 0., -1.,  0.,  2.,  1.,  1.,  0.],
        [ 0., -1., -1.,  1.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  2.,  1.],
        [ 0.,  1.,  0., -1., -1.,  1.,  1.],
        [ 0.,  0.,  2.,  1.,  0.,  0.,  2.]]),
 array([[1., 2., 2., 2., 2., 2., 2.],
        [1., 2., 2., 0., 2., 2., 2.],
        [1., 0., 2., 2., 2., 0., 2.],
        [1., 1., 0., 2., 2., 2., 0.],
        [1., 1., 1., 0., 2., 2., 2.],
        [1., 2., 1., 1., 0., 2., 2.],
        [1., 2., 1., 1., 1., 0., 2.],
        [1.,

In [44]:

Peptide("TAGATA").fitting_alignment(Peptide("GTAGGCTTAAGGTTA"))


(array([[  0.,  -1.,  -2.,  -3.,  -4.,  -5.,  -6.,  -7.,  -8.,  -9., -10.,
         -11., -12., -13., -14., -15.],
        [  0.,  -1.,   0.,  -1.,  -2.,  -3.,  -4.,  -5.,  -6.,  -7.,  -8.,
          -9., -10., -11., -12., -13.],
        [  0.,  -1.,  -1.,   1.,   0.,  -1.,  -2.,  -3.,  -4.,  -5.,  -6.,
          -7.,  -8.,  -9., -10., -11.],
        [  0.,   1.,   0.,   0.,   2.,   1.,   0.,  -1.,  -2.,  -3.,  -4.,
          -5.,  -6.,  -7.,  -8.,  -9.],
        [  0.,   0.,   0.,   1.,   1.,   1.,   0.,  -1.,  -2.,  -1.,  -2.,
          -3.,  -4.,  -5.,  -6.,  -7.],
        [  0.,  -1.,   1.,   0.,   0.,   0.,   0.,   1.,   0.,  -1.,  -2.,
          -3.,  -4.,  -3.,  -4.,  -5.],
        [  0.,  -1.,   0.,   2.,   1.,   0.,  -1.,   0.,   0.,   1.,   0.,
          -1.,  -2.,  -3.,  -4.,  -3.]]),
 array([[1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 0., 2., 2., 2., 2., 2., 

In [45]:

Peptide("GTTGCTAAACCTCCTCATATGTTTTTGCCAAGCTGGACATGCTTCAGGGATAGATAACTCGTCACCTCAGCATCACGCTCGCCGGGGGAGAACCGTTCGTCTGACGTTGCGTCTGGCACCCTACTCCTTTAAGCGTGCGGAAACGAGGGTTTCAACCGGATCTCGCTTTAGTGGCGACCTATATTGGCCACATAGCTTTTCGGGTAAAGGTCAAGAGCCCGCACCCTCGAGCGCCGATTAACAGACGTTCTCTTCACCTACCATGTAGAATACTCAATAAGGAAAGCCAGATAGTCCCGGGGTCTACGCGTAGTCATGCCTGAATTTAGCAGTTATCGATACGTCCAGAAACACGTGTGGACGAGTAGGGCTAAAGGTGCCCTGCAGTATCGCTTTCCATTAAAGTCCGTCGCGCCGGGCTTTTCCGTTATGTGTAGCCAACCGATGGAGGGCCATCAAGCTACAACATGGATTGGAAAACGATGACCAAACGCTCGGACCGATACTCCTAGGGTAGAATAAACAAAGCGGAGGGGGTGAAGATCCCGTTACATTTTAGCCCCGATGCGCATTATTCGGGGGATTTCAGCCATAACGTCTGCCCTCCTGCTGACAATGTGATATGAGGACGCGTGAACAAGTCGTTTCCGACGCACCCGGCTCCGCTAGACCGTAGCCAGCTGCGGTCGCAAAGATAGCAACAGCTCCGCCAACCACGGCGATATTTGAGTCCAGCCATACTTCCTACTTATGACAACACAGCGGTTCTTACCACCAACATAAGAGCTTTGAAGCGGTACGGTCGATTACGCACGCAATGGAGCTGTGTCATTGGAGTGTGATGTAGGCGTTGAGCGCTGCGCCTGAGCGCACTATGTCGATCCCGCCCGCTAATGCTTCTAATTGCACTATGTATAAGACGACGAATGTTCTGACATGCTCTCG").fitting_alignment(Peptide("GTCGTTGTACCGTTGTCCAGTTACAGATTCCGTTGCGGAATGCTGGTAGTGGACATAGCTCTAAATCAGGACGATTCAAC"))



(array([[  0.,  -1.,  -2., ..., -78., -79., -80.],
        [  0.,   1.,   0., ..., -76., -77., -78.],
        [  0.,   0.,   2., ..., -74., -75., -76.],
        ...,
        [  0.,  -1.,   0., ...,  10.,  10.,  12.],
        [  0.,  -1.,  -1., ...,   9.,   9.,  11.],
        [  0.,   1.,   0., ...,   9.,   8.,  10.]]),
 array([[1., 2., 2., ..., 2., 2., 2.],
        [1., 0., 2., ..., 2., 2., 2.],
        [1., 1., 0., ..., 2., 2., 2.],
        ...,
        [1., 2., 0., ..., 0., 1., 1.],
        [1., 2., 1., ..., 1., 1., 1.],
        [1., 0., 2., ..., 0., 1., 1.]]),
 (391, 80),
 array([-80., -78., -76., -74., -72., -70., -68., -66., -64., -62., -60.,
        -58., -56., -54., -52., -50., -48., -46., -44., -42., -40., -38.,
        -36., -34., -34., -32., -30., -30., -28., -27., -26., -24., -24.,
        -23., -21., -21., -20., -18., -16., -14., -12., -12., -10., -10.,
        -10.,  -8.,  -6.,  -4.,  -2.,   0.,   2.,   4.,   6.,   6.,   7.,
          7.,   7.,   8.,  10.,   9.,   8.,   7.

In [46]:

Peptide("CCAT").fitting_alignment(Peptide("AT"), mismatch_penalty=5)

(array([[ 0., -1., -2.],
        [ 0., -1., -2.],
        [ 0., -1., -2.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  2.]]), array([[1., 2., 2.],
        [1., 2., 2.],
        [1., 2., 2.],
        [1., 0., 2.],
        [1., 1., 0.]]), (4, 2), array([-2., -2., -2.,  0.,  2.]), 2.0, AT
 AT)

In [47]:

Peptide("CACGTC").fitting_alignment(Peptide("AT"))[3]

array([-2., -2.,  0.,  0., -1.,  0., -1.])

In [48]:

Peptide("ATCC").fitting_alignment(Peptide("AT"))[3]

array([-2.,  0.,  2.,  1.,  0.])

In [49]:
Peptide("CGT").edit_distance(Peptide("CAGACGGTGACG"))


array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
       [ 1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
       [ 2.,  1.,  1.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
       [ 3.,  2.,  2.,  2.,  2.,  3.,  4.,  5.,  5.,  6.,  7.,  8.,  9.]])

In [50]:
Peptide("CAGACGGTGACG").edit_distance(Peptide("CGT"))


array([[ 0.,  1.,  2.,  3.],
       [ 1.,  0.,  1.,  2.],
       [ 2.,  1.,  1.,  2.],
       [ 3.,  2.,  1.,  2.],
       [ 4.,  3.,  2.,  2.],
       [ 5.,  4.,  3.,  3.],
       [ 6.,  5.,  4.,  4.],
       [ 7.,  6.,  5.,  5.],
       [ 8.,  7.,  6.,  5.],
       [ 9.,  8.,  7.,  6.],
       [10.,  9.,  8.,  7.],
       [11., 10.,  9.,  8.],
       [12., 11., 10.,  9.]])

In [51]:
Peptide("PL").edit_distance(Peptide("MEANLY"))


array([[0., 1., 2., 3., 4., 5., 6.],
       [1., 1., 2., 3., 4., 5., 6.],
       [2., 2., 2., 3., 4., 4., 5.]])

In [52]:
Peptide("GAGA").edit_distance(Peptide("GAT"))


array([[0., 1., 2., 3.],
       [1., 0., 1., 2.],
       [2., 1., 0., 1.],
       [3., 2., 1., 1.],
       [4., 3., 2., 2.]])

In [53]:
Peptide("AC").edit_distance(Peptide("AC"))


array([[0., 1., 2.],
       [1., 0., 1.],
       [2., 1., 0.]])

In [54]:
Peptide("AT").edit_distance(Peptide("G"))


array([[0., 1.],
       [1., 1.],
       [2., 2.]])

In [55]:
Peptide("CAGACCGAGTTAG").edit_distance(Peptide("CGG"))




array([[ 0.,  1.,  2.,  3.],
       [ 1.,  0.,  1.,  2.],
       [ 2.,  1.,  1.,  2.],
       [ 3.,  2.,  1.,  1.],
       [ 4.,  3.,  2.,  2.],
       [ 5.,  4.,  3.,  3.],
       [ 6.,  5.,  4.,  4.],
       [ 7.,  6.,  5.,  4.],
       [ 8.,  7.,  6.,  5.],
       [ 9.,  8.,  7.,  6.],
       [10.,  9.,  8.,  7.],
       [11., 10.,  9.,  8.],
       [12., 11., 10.,  9.],
       [13., 12., 11., 10.]])

In [56]:
Peptide("PLEASANTLY").edit_distance(Peptide("MEANLY"))


array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.],
       [ 1.,  1.,  2.,  3.,  4.,  5.,  6.],
       [ 2.,  2.,  2.,  3.,  4.,  4.,  5.],
       [ 3.,  3.,  2.,  3.,  4.,  5.,  5.],
       [ 4.,  4.,  3.,  2.,  3.,  4.,  5.],
       [ 5.,  5.,  4.,  3.,  3.,  4.,  5.],
       [ 6.,  6.,  5.,  4.,  4.,  4.,  5.],
       [ 7.,  7.,  6.,  5.,  4.,  5.,  5.],
       [ 8.,  8.,  7.,  6.,  5.,  5.,  6.],
       [ 9.,  9.,  8.,  7.,  6.,  5.,  6.],
       [10., 10.,  9.,  8.,  7.,  6.,  5.]])

In [57]:
Peptide("HKREPFDGWAFRKRWDCNPPTKVVALWYRKPPPTCWMAYWIIKMTVNYYNFQMLIAKGSRTECYHISWMIISFQVCTRQGIFRGIVYLSDPKAIEALQMLIGWSWKDALQRVEAWENVAEKNDCKNFLHESMHYYGMCKDEMELTCLYHIWFDYGQKMGWRGWENTHFQSMKHMVIFPNDDWLIEMWSYDSQLETRLIGATSWRNGHWANHMPQDWNMNNWTTCPSKMDWIINQMKTEVLKWKVYSCLFKKIMHISHMYVHTAPYYCHQWMISALRDKILKTCSIMKQMYDQCSKEIQPYDYPMHNQYHPYLYALIMPFEYHNWWDKYAGKINSRGGPQACRVLYMYGPWYRFNLARVTVMTWMLDHAFICWTYDFSSDVAPAKMMISEPPKCFHLTSTGPGPCTGLPTTDACPWHAMFAIAYRCQHESMNFFQCPLAYFCEYRPIRFIQGMCPIHKHRNIARDPCRMLQKCWAWPPPKGYRYPDLQLCSCITPEWGALNQAPVTCSTVFGVCTNMIQKKEIAQHGLLNCKTKFVFSGMDRGSIVQACFQFIVSERIRAVRQLFEINVYWCWTEPYNTYMWIHNHLNNVGWWYVKAVDLAFWPISFDNLPQMHAKWKVVMLRQSKEIHIFVLWKIVDRYADCTGPPEARTETYTGYGEHMVACYDMGDPEHIKELMMKTEVWTLIPGYACFNIQDPNIPDNTCKQPHIIRVQVHFAFHQDDPALEMRVQEKFWICEHYWPGRNPSNWPTMVVKDCGHEIIVIAAKRGYGPRQYVRSPMMQPSTATGPCMTHMPFC").edit_distance(Peptide("HIREMYAQSHRFDTFYPECSWAFRKRMQCNPPTKVVALWYRKPPWMAYWIIFQVLYHISINITCNFMINSIFRLDEALDMLIGWSWKDALQRVCMLAWENVAEKNDQIIPKNFNHEFMPGTKYYGMCKDHCKDRCNKNMEWCQITCLYHIWFDYGQKRQGWRGRENTHFQSMKHMVDWLIESWQYDSQLEDRLIGATNHMPQMWNMNNCPIKMKQHHEMCHTFVLKKIMSVGMYVHTAPYYCHQWMLRDPNKWITFYMILTCSCMKMWNFTCSVEIQPYDYPMHNQYRPNTVWNIYILIMPFEYHNWWDKYKINSRYMYGPWPGARFIKYTLDNFGIDEVTVMTWMLDHAFICWTYDFSSDVAPAKMMISEPGFSSWGAFLKNGLTDARHWWDMKEPWHAMYFYHLPAFSMNFFQCPLAYFCEYRPIRFIQGMCPIHKHRNIARFDPMCMLCCEWCVQRMLPKCWAWPPPKGYRYPDLQLCPCIAPEWGALMQAPWTCVFFVCTMMIQKKEIAQSGLLNCKQCPPGKAFKPHANFMEVFSGGSIVSACFHFIWSERIRRIMQLFEIHVYWCWTEPWNTYMWIKNQLNNVGWWYVKAVDHNFQSLGTVCSARMHAKWKVVMLRQSKEIHIEVLWKIVDRPADCTGPPEARTEQYTGYGEHMVACYDMGDPEHIKELMMKTEVWTLIFSVLISSLNGYACVRLYMVNKKQYCTKQPHIIRVVDHFAFHQDDPAYLDMQEKFWICEHYWPGSNWEHYGKTTSLSLCGHEIIVTQFINFQGTGYGPRSPMMQPSTATGPCMTHMPFC"))


array([[  0.,   1.,   2., ..., 801., 802., 803.],
       [  1.,   0.,   1., ..., 800., 801., 802.],
       [  2.,   1.,   1., ..., 799., 800., 801.],
       ...,
       [793., 792., 791., ..., 325., 326., 327.],
       [794., 793., 792., ..., 326., 325., 326.],
       [795., 794., 793., ..., 327., 326., 325.]])

In [58]:
Peptide("YAQEFRNIPREWRMMNTCIKWRMHSQCTTYHQQTSISHQLNYYEIFCSRVFSPVMSASGCYAISSEWGPHCYITLIGYCCNSRNFLALYFWAFAEDVNLHPWCPMGDMDYTRDGIHLQWPFTADRLHPEFSVYTACMLFDQFLHHVKRPWRYAMIGIRLSKAQSHWVSWTLGDPVLMQKMQTQQAFICNTWSQMWQDVYMWPKSWPIVENGEDDPPEPAANNGMYWMQTGQIDKACEGHTDKCMHYWEDPNNGNIFQEWHEPAAEKSYHWYGHVLQIHGGQDMMRELIVYWFFNWVTWPIFDVWMFEPEQWKHSVQKNERKWHVYNIDGPCYLSIYGPMYLERSLWIWIMVTWSGNSMYFIYIMFHCRCLCIHHTARPGFYVIHVTCGMAYDFWDNAIMGQETCCRPKHPHSQAFTQELHAFNHPHNKMAGYVGMMWGHFPDWCIVYHFSYMKEVFHVHTWDSTYSHSSNWQLDKGQLETNKESRFNQMKERSCFLGFTERSDHFKSQTAQQFYWWKHPTWVLEGPRFEFTQFTEGIDDRTFPSSRGLASTDKCQVPQYGIRWRDQPQAVRQYQVDRGPVYTQKMPNMWIKDWFSPFSARCWMTLGHMYYMRCFTPRFWVANLAGEFVTGDDMKHRDRYEFALCPCHSWCNLYVLLQPFTKVEVDYCSQEYHKFDSPYFRGVNHHYMHFYHASNWGDKTMNMGTCLYFDHCLNFFLTYRGEKAVILWYVIQRTNAWNGIQTIADFEVNRHTQFLPKYVPIYWNWVGHESSMHHVYPCCVTELNMMSRLADMLEQAIILPSERGLLCGGKY").edit_distance(Peptide("HPSAQEFWHIVVREWRMHLTITKWRMCTSYHQQTCPAYFRVISHQLNYYDIFCDRVFSPVMSASGNQGPHKRWQHYITCNFLALVFWAFAETITNKHPWCPMGDMDYQHTDGIHLQWPFTADRLHPEFSVYTLCMLFDYFRHHVKRPWRYAMNRHGIRLSKAQSHWVSWTLGDPVNMQKDQTQVTMWPKSWEDDSLTMFPPEPAANNNMDKAMSHIHPHYYPDPNNGNIFFEWHEPAAEKTARQVYHWYGAVLQIHGGQCKDQVMRELIVYWFFNWVTFYTQKQIWKHSVQLNERKWHVYNIDGPCYHSIYHEASLLQVPHYVTWSGNSMYTGMWLCIHQTARPGAYTMSNRFAPIVDIHVTTGMYVGTMRAHFWDNAIMGQETCSTAHPHKFNHPHNTCFKWMNKMAGYVGMMNIYHFSYMKEVFHVHTWDSLIYYVYSHRSTMSVQCQWQLDKLQLETKLWKESRFNQMKVRSCTPKHFKSQTAQQFYEDPRFPASFLFGWQFTMISFRHPRTFPMSNLLASTDKCQGIHWQAVRFICYWSGPVYTDKMPNMWAQLIPKFKDWFSPFSARCWMTLGHMYMMRCFTPRHFVTGDDMKVMCAKRDRYEFPLSPCHSWCNLYVYLQWFTEYFRGVYHASDKWDEPQVMNMGTCLYFDHCLNFPFQTLLTKRFEKAKILWDVIQYWFGLIWIQTIASFEVNRWHKDTQFLPKAKNPIVWVNHESSTSHVYPNMMSRLADMLEQAIIKKYEKTPSLRNVLLCGGKY"))


array([[  0.,   1.,   2., ..., 761., 762., 763.],
       [  1.,   1.,   2., ..., 760., 761., 762.],
       [  2.,   2.,   2., ..., 759., 760., 761.],
       ...,
       [808., 807., 806., ..., 358., 359., 360.],
       [809., 808., 807., ..., 359., 358., 359.],
       [810., 809., 808., ..., 360., 359., 358.]])

In [59]:
scores, path = Peptide("MEANLY").local_alignment(Peptide("PENALTY"), read_cost_table("PAM250"))
print(scores)
print(path.backtrack)
print(path)

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  4.  0.  0.]
 [ 0.  0.  4.  1.  0.  0.  4.  0.]
 [ 0.  1.  0.  4.  3.  0.  1.  1.]
 [ 0.  0.  2.  2.  4.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  5.  0.]
 [ 0.  0.  0.  0.  0.  5.  7. 15.]]
[[1. 2. 2. 2. 2. 2. 2. 2.]
 [1. 3. 3. 3. 3. 0. 3. 3.]
 [1. 3. 0. 0. 0. 3. 0. 3.]
 [1. 0. 0. 0. 0. 3. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 3.]
 [1. 3. 3. 3. 0. 0. 2. 2.]
 [1. 3. 3. 3. 3. 1. 0. 0.]]
EANL-Y
ENALTY


In [60]:
scores, path = Peptide("CGA").local_alignment(Peptide("G"), read_cost_table("PAM250"))
print(scores)
print(path.backtrack)
print(path)

[[0. 0.]
 [0. 0.]
 [0. 5.]
 [0. 1.]]
[[1. 2.]
 [1. 3.]
 [1. 0.]
 [1. 0.]]
CG
-G


In [61]:
scores, path = Peptide("THFEREYATHNCPPKIPLGYGNHVRNKGMIVHYANIGSFMLVRWVIGMWKCGENPCGASVTRVSVVHHEQDDPLCYDSKFSQKKLETYHCDHGHSIYVKYQLMWIKWDFWERQSRACEESCVCRPSSIILCAKQSDSQLDNATYSRPKIITCNRCNIVGHPKPYCSIMQIWAQRFFCYDIVAKYMVHAEEKYYTESPSAFINYHDSIRTPHQRTSMMCLGQNWYMNNWTDTGIEAAWQFGLELMKCLWDSHSNAFEQQNAKGRQGYDYRFDVNEPARHASKDKSGNRHVYCLQKIMRKWWDTAKCYIQPHWTKPTASVLPAIITLFYWNDKRKTMHTFLNFGGNLTMWYRGAICSWCFTTHIRAYLYYAAGAYAAGNGCSHTSEWAVHRVSRLSNEDMLYEGMRLAVKYYQICDLDTSRNCMCPVQGQPWMSRKIGGVAMDRWPDGCKQCPDVAPHVDALRQTECLEFRELHLAGMKIWKWDRCYNYSWADPMIVNYADSYLQMHIHPVNVDGHCQKVTILWYLSAVEIQWQINDHPRCKVMAALIILWMENLIFRPIHVYVHVMCHMHDGRQETAMTSHIENFCKNDSCRETRTDFMGRVRMGAKQGYPYWIHEHQAWNFKSSWFRCLIKGIMVMGDKQKDNCVCELMGQEHNVLGVVKYMDMMCVIQSAILNASPRHYAENFNIMNEYSAMQTTMLSPKLVFASRNKTTSEAAGNQSNHKIVQSRCHVRMSDLPIGISRYPCSWIKQITNMLVHQIFSQSINLRGMQWVTAEVNVAFVPQIQALIWEKESAQPWFPTMHWGIPWTFTAVNCMWVYLRYNKLRDHQVHAATSVLEEVYHVYQWEAEHEVMGRYGHWNKLFANNHRMRDCVQLPDRPGVVYPHEMINRQDSMICLEPMHP").local_alignment(Peptide("QGIAEWKTRIVMRHIMLARHPIEETPMSFYYYIMLAVSCNPNDANWYIDFNCDVSFQDDSDASRPGCRGMAFAENLYSGGWIWITMHGCVRGWEDWRQEEMRQWYQQTVFKTWLSLGADSYCWTKYFGWVKEATPNEVVPTMYFWFRKPIEGTPDWFWIMKVNVPRSIACAWLYSMGHEPFSDRDDSVDCWSQLRVYCKGVPYCYDAFDQEAVTVRAIPRCPMWYTPMDQYPEVMSWRFMKHDDICEKNMYCCMYHLRTEVTLWDSPSNTGNQVFWHVFEQTNAKGSQGYDYRFDVDEQIAAVDVFHNHVYCWWDTAKYYIQPHWTASVLPAIITLFYWNATRKWLMHTFLNFGGNLTMWYRGAICSWCQTTHIRAYLYAAAIEAYAGNGCSHTSEWAVHRVSRHRDNRSNEDMMKLDPFYEGMRLAVKYYGCQICRLDTSRNCKCPVEWKIGGVAMDRWVDGCKQCPDALVQTAQYFHYWKWDRCGGNMDCPNQMHIQWKVNWWQYGHCKKVTIEWYLQWQINDHPRDIPAEYKVMAALIILWMENLEGRKQHLNYAYNRKTDICWYINTHYVHVHMHRGRQYICHITTVDTAMTSHVHNFCKNDVCRSQDLCLTLCTRPVASRGLQGIYKNGEKSEGNFGWSWLCIEFIFEMSPFECQWGMPNASKPDFWARQGPWMFYHMEEWNQDIFCRHHAHPGVTPNFHEWYPRPRKSTQEIQVTIVETEMQGNKTHIHYSTRFMFIVGYNDKLANFNEYQQNDMIWFNCSARVTLKPNHHAQHGFCSGLWWWPFLPCKGVNTNCAMFSACAPFGYPPVSPVTIHNCENRPHSTRVYEHFYTPYVLQSTHKIALERSTSCIYHRLQQSTGVIWHRFNSIVDQSLAWQCFLCASWPHQCAVDLKWWWDITFMCTVTKTPDSQGGC"), read_cost_table("PAM250"))
print(scores)
print(path.backtrack)
print(path)
print(scores.max())

[[   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 [   0.    3.    0. ...    0.    0.    0.]
 ...
 [   0.    0.    0. ... 1054. 1049. 1053.]
 [   0.    3.    0. ... 1061. 1056. 1051.]
 [   0.    0.    3. ... 1066. 1061. 1056.]]
[[1. 2. 2. ... 2. 2. 2.]
 [1. 3. 0. ... 0. 0. 3.]
 [1. 0. 3. ... 3. 3. 3.]
 ...
 [1. 3. 3. ... 2. 1. 1.]
 [1. 0. 3. ... 2. 2. 2.]
 [1. 0. 0. ... 0. 2. 2.]]
YATHNCPPKIPLGYGNHVRN--KGMIVHYANIGSFMLVRWVIGMWKCGENPCGASVTRVSVVHHEQDDPLCYDSKFSQKKLETYHCDHGHSIYVKYQLMWIKWDF-WERQSRACEESCVCRPSSIILCAKQSDSQLD---NATYSRPKIITCNR-CNIVGHPKPYCSIMQ-I--WAQ-RFFCYDIVAKYMVHAEEKYYTES-PS-A--F--IN-YHDSIRTPHQRTSMMCLGQNWY--MNNWTDTGIEAAWQFGLEL-MKCLWDSHSNAFEQQNAKGRQGYDYRFDVNEPARHASKDKSGNRHVYCLQKIMRKWWDTAKCYIQPHWTKPTASVLPAIITLFYWNDKRK-TMHTFLNFGGNLTMWYRGAICSWCFTTHIRAYLYYAA-GAYAAGNGCSHTSEWAVHRVSRL----SNEDM--L---YEGMRLAVKYY--QICDLDTSRNCMCPVQGQPWMSRKIGGVAMDRWPDGCKQCPDVAPHVDALRQTECLEFRELHLAGMKIWKWDRCYNYSWADPMIVNYADSYLQMHIH-PVNV-D-GHCQKVTILWYLSAVEIQWQINDHPR-----CKVMAA

In [62]:
scores, path = Peptide("CGA").global_alignment(Peptide("G"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)

[[  0.  -5.]
 [ -5.  -3.]
 [-10.   1.]
 [-15.  -4.]]
[[1. 2.]
 [1. 0.]
 [1. 0.]
 [1. 1.]]
CGA
-G-


In [63]:
scores, path = Peptide("G").global_alignment(Peptide("CGA"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)

[[  0.  -5. -10. -15.]
 [ -5.  -3.   1.  -4.]]
[[1. 2. 2. 2.]
 [1. 0. 0. 2.]]
-G-
CGA


In [64]:
scores, path = Peptide("G").global_alignment(Peptide("ACATACGATG"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[  0.  -5. -10. -15. -20. -25. -30. -35. -40. -45. -50.]
 [ -5.   0.  -5. -10. -15. -20. -25. -24. -29. -34. -39.]]
[[1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 0. 2. 2. 2. 2. 2. 0. 2. 2. 2.]]
------G---
ACATACGATG
-39.0


In [65]:
scores, path = Peptide("ACT").global_alignment(Peptide("ACG"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[  0.  -5. -10. -15.]
 [ -5.   4.  -1.  -6.]
 [-10.  -1.  13.   8.]
 [-15.  -6.   8.  11.]]
[[1. 2. 2. 2.]
 [1. 0. 2. 2.]
 [1. 1. 0. 2.]
 [1. 1. 1. 0.]]
ACT
ACG
11.0


In [66]:
scores, path = Peptide("AG").global_alignment(Peptide("AT"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[  0.  -5. -10.]
 [ -5.   4.  -1.]
 [-10.  -1.   2.]]
[[1. 2. 2.]
 [1. 0. 2.]
 [1. 1. 0.]]
AG
AT
2.0


In [67]:
scores, path = Peptide("CA").global_alignment(Peptide("TCA"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[  0.  -5. -10. -15.]
 [ -5.  -1.   4.  -1.]
 [-10.  -5.  -1.   8.]]
[[1. 2. 2. 2.]
 [1. 0. 0. 2.]
 [1. 0. 1. 0.]]
-CA
TCA
8.0


In [68]:
scores, path = Peptide("CC").global_alignment(Peptide("TTTTCCTT"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[  0.  -5. -10. -15. -20. -25. -30. -35. -40.]
 [ -5.  -1.  -6. -11. -16. -11. -16. -21. -26.]
 [-10.  -6.  -2.  -7. -12.  -7.  -2.  -7. -12.]]
[[1. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 0. 2. 2. 2. 0. 2. 2. 2.]
 [1. 1. 0. 2. 2. 0. 0. 2. 2.]]
----CC--
TTTTCCTT
-12.0


In [69]:
scores, path = Peptide("T").global_alignment(Peptide("ACAGATTAG"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[  0.  -5. -10. -15. -20. -25. -30. -35. -40. -45.]
 [ -5.   0.  -5. -10. -15. -20. -20. -25. -30. -35.]]
[[1. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 0. 2. 2. 2. 2. 0. 2. 2. 2.]]
-----T---
ACAGATTAG
-35.0


In [70]:
scores, path = Peptide("ACAGATTAG").global_alignment(Peptide("T"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[  0.  -5.]
 [ -5.   0.]
 [-10.  -5.]
 [-15. -10.]
 [-20. -15.]
 [-25. -20.]
 [-30. -20.]
 [-35. -25.]
 [-40. -30.]
 [-45. -35.]]
[[1. 2.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
ACAGATTAG
-----T---
-35.0


In [71]:
scores, path = Peptide("T").global_alignment(Peptide("ACAGATTAG"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[  0.  -5. -10. -15. -20. -25. -30. -35. -40. -45.]
 [ -5.   0.  -5. -10. -15. -20. -20. -25. -30. -35.]]
[[1. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 0. 2. 2. 2. 2. 0. 2. 2. 2.]]
-----T---
ACAGATTAG
-35.0


In [72]:
scores, path = Peptide("MEANLY").global_alignment(Peptide("PLEASANTLY"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)

[[  0.  -5. -10. -15. -20. -25. -30. -35. -40. -45. -50.]
 [ -5.  -2.  -3.  -8. -13. -18. -23. -28. -33. -38. -43.]
 [-10.  -6.  -5.   2.  -3.  -8. -13. -18. -23. -28. -33.]
 [-15. -11.  -7.  -3.   6.   1.  -4.  -9. -14. -19. -24.]
 [-20. -16. -12.  -7.   1.   7.   2.   2.  -3.  -8. -13.]
 [-25. -21. -12. -12.  -4.   2.   6.   1.   1.   1.  -4.]
 [-30. -26. -17. -14.  -9.  -3.   1.   4.  -1.   0.   8.]]
[[1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2.]
 [1. 1. 0. 1. 0. 2. 2. 2. 2. 2. 2.]
 [1. 1. 1. 0. 1. 0. 2. 0. 2. 2. 2.]
 [1. 1. 0. 1. 1. 1. 0. 2. 0. 0. 2.]
 [1. 1. 1. 0. 1. 1. 1. 0. 2. 0. 0.]]
-MEA--N-LY
PLEASANTLY


In [73]:
scores, path = Peptide("ASGEEDN").global_alignment(Peptide("GPPPWLSEEQN"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)

[[  0.  -5. -10. -15. -20. -25. -30. -35. -40. -45. -50. -55.]
 [ -5.   0.  -5. -10. -15. -20. -25. -29. -34. -39. -44. -49.]
 [-10.  -5.  -1.  -6. -11. -16. -21. -21. -26. -31. -36. -41.]
 [-15.  -4.  -6.  -3.  -8. -13. -18. -21. -23. -28. -33. -36.]
 [-20.  -9.  -5.  -7.  -4.  -9. -14. -18. -16. -18. -23. -28.]
 [-25. -14. -10.  -6.  -8.  -7. -12. -14. -13. -11. -16. -21.]
 [-30. -19. -15. -11.  -7. -12. -11. -12. -12. -11. -11. -15.]
 [-35. -24. -20. -16. -12. -11. -15. -10. -12. -12. -11.  -5.]]
[[1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 0. 2. 2. 2. 2. 2. 0. 2. 2. 2. 2.]
 [1. 1. 0. 2. 2. 2. 2. 0. 2. 2. 2. 2.]
 [1. 0. 1. 0. 2. 2. 2. 0. 0. 2. 2. 0.]
 [1. 1. 0. 0. 0. 2. 2. 0. 0. 0. 2. 2.]
 [1. 1. 1. 0. 0. 0. 2. 0. 0. 0. 2. 2.]
 [1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]]
AS----GEEDN
GPPPWLSEEQN


In [74]:
scores, path = Peptide("GAGA").global_alignment(Peptide("GAT"), read_cost_table("BLOSUM62"), 1)
print(scores)
print(path.backtrack)
print(path)


[[ 0. -1. -2. -3.]
 [-1.  6.  5.  4.]
 [-2.  5. 10.  9.]
 [-3.  4.  9.  8.]
 [-4.  3.  8.  9.]]
[[1. 2. 2. 2.]
 [1. 0. 2. 2.]
 [1. 1. 0. 2.]
 [1. 1. 1. 1.]
 [1. 1. 1. 0.]]
GAGA
GA-T


In [75]:
scores, path = Peptide("GAT").global_alignment(Peptide("GAGA"), read_cost_table("BLOSUM62"), 1)
print(scores)
print(path.backtrack)
print(path)


[[ 0. -1. -2. -3. -4.]
 [-1.  6.  5.  4.  3.]
 [-2.  5. 10.  9.  8.]
 [-3.  4.  9.  8.  9.]]
[[1. 2. 2. 2. 2.]
 [1. 0. 2. 2. 2.]
 [1. 1. 0. 2. 2.]
 [1. 1. 1. 1. 0.]]
GA-T
GAGA


In [76]:
scores, path = Peptide("ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA").global_alignment(Peptide("ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[ 0.000e+00 -5.000e+00 -1.000e+01 ... -3.715e+03 -3.720e+03 -3.725e+03]
 [-5.000e+00  4.000e+00 -1.000e+00 ... -3.706e+03 -3.711e+03 -3.716e+03]
 [-1.000e+01 -1.000e+00  8.000e+00 ... -3.697e+03 -3.702e+03 -3.707e+03]
 ...
 [-3.660e+03 -3.651e+03 -3.642e+03 ...  1.545e+03  1.540e+03  1.535e+03]
 [-3.665e+03 -3.656e+03 -3.647e+03 ...  1.540e+03  1.551e+03  1.546e+03]
 [-3.670e+03 -3.661e+03 -3.652e+03 ...  1.535e+03  1.546e+03  1.555e+03]]
[[1. 2. 2. ... 2. 2. 2.]
 [1. 0. 2. ... 2. 2. 2.]
 [1. 1. 0. ... 2. 2. 2.]
 ...
 [1. 1. 1. ... 0. 2. 2.]
 [1. 1. 1. ... 1. 0. 2.]
 [1. 1. 1. ... 1. 1. 0.]]
ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVP--LNP-QMQTVFQNRGLDRVTVKTDC----HDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLY-KKTEYC-LVR---HS-NCC-HCDWDQIIHTCAF-----IAFLDLYQRHWPMCDKLY------C---HFHHSWFCRNQEMSM--D---W---------N--Q---WFP-------WDSV-P-RANCLE-EGALIALYAGIWANSMKRDMKTDHACT--VRLIYVCELHAWLK-YCYTSINML-CGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQ------GICPVERKSHCEIYAKDLLSFESCCSQK-QNCYTDNWGLEYRLFFQHIQMECTD

In [77]:
scores, path = Peptide("RAYRFVVANGFRNATIWCKPFYTRFKHSHMGYHATYNVKCWGDTPFGMAHSILGCCNMYYSFRCKWHWCIDTMAPSKVQLARILSHTTSYWHLCSGNDRWYDCCIRKWEIDQVHGHTLMMFFYMRLQCSDRHDLCIMWWSNDNNNWGVVIYNIKHRILLKAVPRQWDYMCVVYRSCHWLLPKRYDPAYLIFCWDNVQQCFEHPATEQECDICADDVYSRDVICKDPYMGTDKGHQWHMQYSPDLVYMMWHKNNHEPNFRKGKYFYDEMNAWLDCSDMTMPRGYVAIVQVSYAKKGIETPYVQGDFGSSRSFKIQKSNAYFKQPQSISNANIQYTSAIKQCHRQIWDYFWPNCCLGEHDVCQWNESLGWREPYIFRATAVSFNMQFAILISGKGPNLDPWDKGFLLPRSYCKFEYFCHKSITCKKYAYVPLMYELNFIKNNTRMVRCSNKDKKLHLRGFKDPENMPTLWALENQPSYELCYDFMYKWFTRMQNEPNNQIRLLQGQTVWKIIHQFLRRFKHFAHINMCFMRREIEYQTPWCLISCTANRAEHCMLLEFSTFSAAFPGFCWICEYEGLMRRGFVSPYTHKGLYGPVQAATGNEIYSKNDHCVFWWDTPEHYQHINVDQESNAGWKSCTAVGHYNWYEKWCWSTDGQYILRCWRNQCIEVSSQCKCWKRCCIQLRNLWGTHPYDRPRAGGPGEQIWGCYICRQWEMAAFKWQEDMTWSWWGMYFPKWCQLDLKNWLWTCMFEANINFIRKRHIYAMVDSDYKHRYRNTHQMSLSDYFCYVHEVNCVNTYFKK").global_alignment(Peptide("YCLHMVKIIWCKDLYTRFKHSHMGYHATYLLTAICIFVFKCWGDTPFGCCNMYFKTSCDWDNFRCKWHWCRKVQIARILSTSYWHLVPGDQVGCCIRKWRIDKVHGHWLMMFFYGYTHCHQIRLQSDLWEWHDSCIMWWSNDNNNWGVVTHVWSQPIYNTIDICWWNRRILLKAVPVQWDYMCIVIDIFPVSCWLLPKRYDPAYLIFKMMETHDVCSFLGYEHPDVIMWTDKGHQWHMQVWQANLEHYSPDLVYMMYINFSKENNHETAPRNCGPWHARMGKYFYDEMNAWLDCSAGIETPYVQGDCIQSSNAYFKQPQYTSAIKQCHRQIWDYFWPNCCPGEHDECQWNESLFGREPYIFRATAVSDNMQFAILISGKGPNSDPWDKRCDYRWCEPFLLHRSYGHFACIFEYFNYAYVYEINMIKNNTRMVKDPENMPNLMANQPSYEMQNEPHPYTCGCTVWKITHCFLRRLKHFAHINEYSCTAEFSNFSAAFPGFCWICEYEGPIWKGLYGPVQAATGNEIYSKNDHCVFWWDHINVDQESNAGWQFTTDSCTAVGHMNWYEINYQTDGQGILRVWRNQCLEVELECIQETNLWGTHPYDRPRVSNNSHGGPGNQIWGCYIKRQWEMAAFKWQEDMTWYWWGMYFPKWCQLDLKNWYKHTLGLVTCMHDFWMRINFIRKPHIRAMVDSDQMSLSDYKCYVHEGTCVCTYWKF"), read_cost_table("BLOSUM62"))
print(scores)
print(path.backtrack)
print(path)
print(scores[-1][-1])

[[ 0.000e+00 -5.000e+00 -1.000e+01 ... -3.570e+03 -3.575e+03 -3.580e+03]
 [-5.000e+00 -2.000e+00 -7.000e+00 ... -3.560e+03 -3.565e+03 -3.570e+03]
 [-1.000e+01 -7.000e+00 -2.000e+00 ... -3.551e+03 -3.556e+03 -3.561e+03]
 ...
 [-3.980e+03 -3.968e+03 -3.954e+03 ...  1.747e+03  1.742e+03  1.742e+03]
 [-3.985e+03 -3.973e+03 -3.959e+03 ...  1.742e+03  1.752e+03  1.747e+03]
 [-3.990e+03 -3.978e+03 -3.964e+03 ...  1.737e+03  1.747e+03  1.749e+03]]
[[1. 2. 2. ... 2. 2. 2.]
 [1. 0. 2. ... 2. 2. 2.]
 [1. 1. 0. ... 2. 2. 2.]
 ...
 [1. 1. 1. ... 0. 2. 0.]
 [1. 1. 1. ... 1. 0. 2.]
 [1. 1. 1. ... 1. 1. 0.]]
RAYRFVVANGFRNATIWCKPFYTRFKHSHMGYHATY--N-----V-KCWGDTPFGMAHSILGC-CNMYYSFRCKWHWCIDTMAPSKVQLARILSHTTSYWHLCSGNDRWYDCCIRKWEIDQVHGHTLMMFFY-------MRLQCSD--R-HDLCIMWWSNDNNNWGVV-------IYN-IK----HR-ILLKAVPRQWDYMCVV---YR-SCHWLLPKRYDPAYLIFCWDNVQQCFEHPATEQECDICADDVYSRDVICKDPYMGTDKGHQWHMQ--------YSPDLVYMMW-H--K-NNHE--P-N---F--RKGKYFYDEMNAWLDCSDMTMPRGYVAIVQVSYAKKGIETPYVQGDFGSSRSFKIQKSNAYFKQPQSISNANIQYTSAIKQCHRQI